In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'Zuckerberg-Testimony'
device = 'cuda'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('%100')

['%', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['Text']
y = data['Person']

In [10]:
X.to_csv('./X.csv')
y.to_csv('./y.csv')

In [11]:
words = []
data = []
idx = 0
labels = {}
labels_r = {}

In [12]:
for yb in y:
    if yb not in list(labels.keys()):
        idx += 1
        labels[yb] = idx
        labels_r[idx] = yb

In [13]:
for Xb,yb in tqdm(zip(X,y)):
    Xb = tokenize(Xb)
    new_X = []
    for X_batch in Xb:
        new_X.append(stem(X_batch))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[yb]+1,len(labels))[labels[yb]]
    ])

In [14]:
words = sorted(set(words))

In [15]:
np.random.shuffle(data)

In [16]:
data[0][1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
X = []
y = []

In [18]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

In [19]:
from sklearn.model_selection import *

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [21]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [24]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.iters = 10
        self.activation = ReLU()
        self.hidden = 512
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear2bn = BatchNorm1d(self.hidden)
        self.output = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.output(preds)

In [25]:
model = Model().to(device)

In [26]:
criterion = MSELoss()

In [27]:
optimizer = Adam(model.parameters(),lr=0.001)

In [28]:
batch_size = 32

In [29]:
epochs = 100

In [30]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

In [31]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.iters = 10
        self.activation = ReLU()
        self.hidden = 512
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear2bn = BatchNorm1d(self.hidden)
        self.output = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.output(preds)
        return preds

In [32]:
model = Model().to(device)

In [33]:
criterion = MSELoss()

In [34]:
optimizer = Adam(model.parameters(),lr=0.001)

In [35]:
batch_size = 32

In [36]:
epochs = 100

In [37]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()